In [1]:
import cv2
import mediapipe as mp

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [3]:
GESTURES = {
    'FIST': 'Fist',
    'PALM': 'Palm',
    'THUMBS_UP': 'Thumbs Up',
    'PEACE': 'Peace'
}

FINGER_TIPS = [4, 8, 12, 16, 20]
FINGER_MCP = [2, 5, 9, 13, 17]


def detect_gesture(landmarks):
    """
    Simple rule-based gesture detection based on finger extension.
    landmarks: list of normalized landmark tuples (x, y, z)
    """
    # Determine if fingers are open or closed
    fingers_open = []
    # Thumb: compare tip x to MCP x (right hand assumption)
    if landmarks[FINGER_TIPS[0]].x < landmarks[FINGER_MCP[0]].x:
        fingers_open.append(True)
    else:
        fingers_open.append(False)

    # Other fingers: compare tip y to PIP y
    for tip_idx, mcp_idx in zip(FINGER_TIPS[1:], FINGER_MCP[1:]):
        if landmarks[tip_idx].y < landmarks[mcp_idx].y:
            fingers_open.append(True)
        else:
            fingers_open.append(False)

    # Count open fingers
    count_open = sum(fingers_open)

    # Determine gesture
    if count_open == 0:
        return GESTURES['FIST']
    elif count_open == 5:
        return GESTURES['PALM']
    elif fingers_open[0] and not any(fingers_open[1:]):
        return GESTURES['THUMBS_UP']
    elif fingers_open[1] and fingers_open[2] and not (fingers_open[0] or fingers_open[3] or fingers_open[4]):
        return GESTURES['PEACE']
    else:
        return None

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip and convert BGR to RGB
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process image
        results = hands.process(image)

        # Draw and annotate
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                gesture = detect_gesture(hand_landmarks.landmark)
                if gesture:
                    # Display gesture
                    cv2.putText(image, gesture, (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Hand Gesture Detection', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()